In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from panther.nn import SKConv2d


class CNN(nn.Module):
    def __init__(self, use_custom=False):
        super(CNN, self).__init__()
        conv_layer = SKConv2d if use_custom else nn.Conv2d
        self.conv1 = conv_layer(3, 16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
def get_dataloader(batch_size=64):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    trainloader = DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    testset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform
    )
    testloader = DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=2
    )
    return trainloader, testloader


# Training function
def train_model(model, trainloader, device, epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")


# Compare custom vs PyTorch Conv2D
def compare_models():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainloader, testloader = get_dataloader()

    # Train model with PyTorch Conv2D
    print("Training PyTorch Conv2D model...")
    # pytorch_model = CNN(use_custom=False)
    # train_model(pytorch_model, trainloader, device)

    # Train model with Custom Conv2D
    print("Training Custom Conv2D model...")
    custom_model = CNN(use_custom=True)
    train_model(custom_model, trainloader, device)

    # Compare weight updates
    print("Weight difference after training:")
    for pytorch_param, custom_param in zip(
        pytorch_model.parameters(), custom_model.parameters()
    ):
        print(torch.norm(pytorch_param - custom_param).item())


In [3]:
compare_models()
# 1:26 m:sec for normal conv2d
# 1:38 m:sec for custom conv2d

Files already downloaded and verified
Files already downloaded and verified
Training PyTorch Conv2D model...
Epoch 1, Loss: 1.4389274607381553
Epoch 2, Loss: 1.1468068460369354
Epoch 3, Loss: 1.0223032017345624
Epoch 4, Loss: 0.9241124717018488
Epoch 5, Loss: 0.8470413618532898
Training Custom Conv2D model...


IndexError: tuple index out of range

In [ ]:
compare_models()

Files already downloaded and verified
Files already downloaded and verified
Training PyTorch Conv2D model...
Epoch 1, Loss: 1.420482767969751
Epoch 2, Loss: 1.1337713077854927
Epoch 3, Loss: 1.0030834277724976
Epoch 4, Loss: 0.9096477385372153
Epoch 5, Loss: 0.8312261295897881
Training Custom Conv2D model...
{'dtype': None, 'device': None}


KeyboardInterrupt: 